In [ ]:
# This model is optimized for GPU

In [1]:
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

from ogb.graphproppred import PygGraphPropPredDataset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.data import DataLoader

from logger import Logger
from tqdm.notebook import trange, tqdm

In [2]:
device_args = 0 #default
device = f'cuda:{device_args}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [3]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())
split_idx = dataset.get_idx_split()
data = dataset[0]

data.adj_t = data.adj_t.to_symmetric()
data = data.to(device)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [4]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)

In [5]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [6]:
num_layers = 3
hidden_channels = 256
dropout = 0.5
lr = 0.1
epochs = 500
runs = 2
log_steps = 50

model = GCN(data.num_features, hidden_channels, dataset.num_classes, num_layers, dropout).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [8]:
for run in range(runs):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in tqdm(range(1, 1 + epochs)):
        loss = train(model, data, train_idx, optimizer)
        result = test(model, data, split_idx, evaluator)
        # logger.add_result(run, result)

        if epoch % log_steps == 0:
            train_acc, valid_acc, test_acc = result
            print(f'Run: {run + 1:02d}, '
                  f'Epoch: {epoch:02d}, '
                  f'Loss: {loss:.4f}, '
                  f'Train: {100 * train_acc:.2f}%, '
                  f'Valid: {100 * valid_acc:.2f}% '
                  f'Test: {100 * test_acc:.2f}%')

  0%|          | 0/500 [00:00<?, ?it/s]

Run: 01, Epoch: 50, Loss: 1.3879, Train: 62.60%, Valid: 62.36% Test: 60.47%
Run: 01, Epoch: 100, Loss: 1.1788, Train: 68.24%, Valid: 67.96% Test: 67.23%
Run: 01, Epoch: 150, Loss: 1.0871, Train: 70.58%, Valid: 69.78% Test: 68.63%
Run: 01, Epoch: 200, Loss: 1.0335, Train: 71.85%, Valid: 70.62% Test: 69.74%
Run: 01, Epoch: 250, Loss: 0.9944, Train: 72.65%, Valid: 71.11% Test: 69.79%
Run: 01, Epoch: 300, Loss: 0.9695, Train: 73.15%, Valid: 70.98% Test: 69.29%
Run: 01, Epoch: 350, Loss: 0.9432, Train: 73.56%, Valid: 71.58% Test: 70.56%
Run: 01, Epoch: 400, Loss: 0.9254, Train: 74.09%, Valid: 71.94% Test: 71.13%
Run: 01, Epoch: 450, Loss: 0.9082, Train: 74.33%, Valid: 71.69% Test: 70.52%
Run: 01, Epoch: 500, Loss: 0.8953, Train: 75.02%, Valid: 71.75% Test: 70.49%


  0%|          | 0/500 [00:00<?, ?it/s]

Run: 02, Epoch: 50, Loss: 1.4113, Train: 62.15%, Valid: 62.27% Test: 60.17%
Run: 02, Epoch: 100, Loss: 1.1888, Train: 68.27%, Valid: 68.06% Test: 66.98%
Run: 02, Epoch: 150, Loss: 1.1086, Train: 70.42%, Valid: 69.31% Test: 67.77%
Run: 02, Epoch: 200, Loss: 1.0562, Train: 71.37%, Valid: 70.39% Test: 69.06%
Run: 02, Epoch: 250, Loss: 1.0280, Train: 72.16%, Valid: 70.95% Test: 69.74%
Run: 02, Epoch: 300, Loss: 0.9917, Train: 72.58%, Valid: 70.69% Test: 68.59%
Run: 02, Epoch: 350, Loss: 0.9720, Train: 73.04%, Valid: 70.29% Test: 68.70%
Run: 02, Epoch: 400, Loss: 0.9558, Train: 73.12%, Valid: 71.28% Test: 70.31%
Run: 02, Epoch: 450, Loss: 0.9374, Train: 73.66%, Valid: 71.11% Test: 70.05%
Run: 02, Epoch: 500, Loss: 0.9279, Train: 74.14%, Valid: 71.76% Test: 70.95%
